In [ ]:
# autoreload modules
%load_ext autoreload
%autoreload 2
%matplotlib ipympl

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

from b2aiprep.dataset import VBAIDataset

We have reformatted the data into a [BIDS](https://bids-standard.github.io/bids-starter-kit/folders_and_files/folders.html)-like format. The data is stored in a directory with the following structure:

```
data/
    sub-01/
        ses-01/
            beh/
                sub-01_ses-01_questionnaire.json
    sub-02/
        ses-01/
            beh/
                sub-01_ses-01_questionnaire.json
    ...
```

i.e. the data reflects a subject-session-datatype hierarchy. The `beh` subfolder, for behavioural data, was chosen to store questionnaire data as it is the closest approximation to the data collected.

We have provided utilities which load in data from the BIDS-like dataset structure. The only input needed is the folder path which stores the data.

In [ ]:
# TODO: allow user to specify input folder input
dataset = VBAIDataset('../output')

In [ ]:
# every user has a sessionschema which we can get info for the users from
qs = dataset.load_questionnaires('sessionschema')
q_dfs = []
for i, questionnaire in enumerate(qs):
    # get the dataframe for this questionnaire
    df = dataset.questionnaire_to_dataframe(questionnaire)
    df['dataframe_number'] = i
    q_dfs.append(df)
    i += 1

# concatenate all the dataframes
sessionschema_df = pd.concat(q_dfs)
sessionschema_df = pd.pivot(sessionschema_df, index='dataframe_number', columns='linkId', values='valueString')
sessionschema_df

The above session schema gives us some information about each "session" which occurred for each participant. In this context, a session is a clinical encounter where the participant answers a number of questions and has a number of audio recordings taken. The above dataframe has

- `record_id` - a unique identifier for the individual
- `session_duration` - the length in seconds of the session
- `session_id` - a unique identifier for the session
- `session_is_control_participant` - "Yes" or "No" indicating whether the participant is a control
- `session_site` - The site of data collection: one of "Mt. Sinai", "VUMC", "USF", "WCM", or "MIT"
- `session_status` - This indicates if the session was completed. Since we are using the deidentified release data, every row should be "Completed".

Note what happened in the above cell to get us this `sessionschema_df` dataframe:

1. We determined the name of the QuestionnaireResponse that we wanted to load in. In this case it was `'sessionschema'`.
2. We called `load_questionnaires(questionnare_name)` with the name of the questionnaire. This function returned a dictionary: the keys are the participant identifiers (`record_id`), and the values were a set of `QuestionnaireResponse` objects which matched the questionnaire name.
3. We then iterated through each participant's `QuestionnaireResponse` objects and converted them to pandas dataframes using `questionnaire_to_dataframe()`.
4. We concatenated the resultant dataframes together.
5. The concatenated dataframe was in "long" form, with three columns: the participant identifier (`record_id`), a unique identifier for each question (`linkId`), and the response given (`valueString`). We used `pd.pivot()` in order to transform this into a "wide" dataframe, with one column per question.

The above process is involved, but consistent across questionnaires. For convenience, the `load_and_pivot_questionnaire` helper function automatically performs these tasks for a given questionnaire. Let's try it with the demographics dataframe.

In [ ]:
demographics_df = dataset.load_and_pivot_questionnaire('qgenericdemographicsschema')
demographics_df.head()

We can iterate through a couple of columns and summarize the data.

In [ ]:
for column in ['children', 'country', 'ethnicity', 'gender_identity', 'grandparent', 'housing_status']:
    print(demographics_df[column].value_counts(), end='\n\n')

## Finding data

The dataset functions are convenient, but they require some prior knowledge: we needed to know that `sessionschema` was the name for the questionnaire with session information. We also needed to know that `demographics` was the name for the questionnaire where general demographics were collected. For convenience, the dataset object has another method which tells you all of the questionnaire names available. It accomplishes this by iterating through every file of the BIDS dataset. Note that this can be an expensive operation! Luckily, if there are less than 10,000 files, it goes pretty fast. Let's try it out.

In [ ]:
questionnaire_types = dataset.list_questionnaire_types()
for q in questionnaire_types:
    print(q)

The above lists out the possible inputs to questionnaire types. Note that some questionnaires are only asked once per subject, such as eligibility. We can limit our list to only these questionnaires:

In [ ]:
questionnaire_types = dataset.list_questionnaire_types(subject_only=True)
for q in questionnaire_types:
    print(q)

Above we see that `eligibility`, `enrollment`, `participant` and a number of disease specific questionnaires are only ever asked once for the participant. As before, we can use `load_and_pivot_questionnaire()` to load in this data into a convenient to use dataframe.

In [ ]:
eligibility_df = dataset.load_and_pivot_questionnaire('eligibility')
eligibility_df.head()

## Participants dataframe

We can get a dataframe summarizing the participants in the dataset.

In [ ]:
participant_df = dataset.load_and_pivot_questionnaire('participant')
participant_df.head()

In [ ]:
# bar chart of participant by enrollment institution
plt.figure(figsize=(10, 5))
participant_df['enrollment_institution'].value_counts().plot(kind='bar')
plt.show()

## Session data

Load in the `QuestionnaireResponse` objects for the session schema.

In [ ]:
session_schema = dataset.load_questionnaires('sessionschema')

# Each element is a QuestionnaireResponse, a pydantic object
# you can serialize it to a python dictionary with .dict()
# and to a json with .json()
# otherwise attributes are accessible like any other python object
questionnaire = session_schema[0]

print(f'FHIR ID: {questionnaire.id}')
print(f'First item response: {questionnaire.item[0]}')
print('\nAbridged questionnaire as JSON:')
# only print the first 600 characters of the JSON for brevity
print(questionnaire.json(indent=2)[:660], end='\n...\n')

In [ ]:
# helper function which loads in questionnaires ending with a specific schema name as a dataframe
session_df = dataset.load_and_pivot_questionnaire('sessionschema')
session_df.head()

We can look at a specific questionnaire which is collected for each session in a similar way.

In [ ]:
session_confounders = dataset.load_questionnaires('qgenericconfoundersschema')
questionnaire = session_confounders[0]

# Each element is a QuestionnaireResponse, a pydantic object
# you can serialize it to a python dictionary with .dict()
# and to a json with .json()
# otherwise attributes are accessible like any other python object
print(questionnaire.json(indent=2))

## Acoustic tasks

Let's look at the acoustic tasks now. Acoustic task files are organized in the following way:

```
data/
    sub-01/
        ses-01/
            beh/
                sub-01_ses-01_task-<TaskName>_acoustictaskschema.json
                sub-01_ses-01_task-<TaskName>_rec-<TaskName>-1_recordingschema.json
                ...
```

where `TaskName` is the name of the acoustic task, including:

* `Audio-Check`
* `Cinderalla-Story`
* `Rainbow-Passage`

etc. The audio tasks are listed currently in b2aiprep/prepare.py:_AUDIO_TASKS.

In [ ]:
acoustic_tasks = dataset.load_questionnaires('acoustictaskschema')
print(acoustic_tasks[0].json(indent=2))

Each row in the above corresponds to a different acoustic task: an audio check, prolonged vowels, etc. The `value_counts()` method for pandas DataFrames lets us count all the unique values for a column.

In [ ]:
acoustic_tasks_df = dataset.load_and_pivot_questionnaire('acoustictaskschema')
acoustic_tasks_df.head()

Above will list out all of the acoustic tasks, as every acoustic task is associated with a single "acoustictaskschema" `QuestionnaireResponse` object.

In [ ]:
acoustic_tasks_df['acoustic_task_name'].value_counts()

## Audio

In [ ]:
from pathlib import Path
import numpy as np
from b2aiprep.process import Audio, specgram
import IPython.display as Ipd

# configuration options
win_length = 512
hop_length = 256

base_path = Path('/Users/alistairewj/git/b2aiprep/output/')
audio_file = base_path.joinpath(
    'sub-1f9475bb-f13b-4f68-969b-28f20455b3e7',
    'ses-CB8A74EE-0C8C-4B15-B322-D93A79ADB40A',
    'audio',
    'sub-1f9475bb-f13b-4f68-969b-28f20455b3e7_ses-CB8A74EE-0C8C-4B15-B322-D93A79ADB40A_Audio-Check_rec-Audio-Check-1.wav'
)
audio = Audio.from_file(str(audio_file))
audio = audio.to_16khz()

# convert to uint32 - probably should use the bits_per_sample from the original metadata!
signal = audio.signal.squeeze()
signal = (np.iinfo(np.uint32).max * (signal - signal.min())) / (signal.max() - signal.min())
# display a widget to play the audio file
Ipd.display(Ipd.Audio(data=signal, rate=audio.sample_rate))

We can also calculate a spectrogram of the data to visualize the frequency components over time. A spectrogram is essentially a frequency spectrum repeated `N` times spaced out throughout the original audio.

In [ ]:
win_length = 30
hop_length = 10
nfft = 512
features_specgram = specgram(audio, win_length=win_length, hop_length=hop_length, n_fft=nfft)
features_specgram = features_specgram.numpy()
# convert to db
log_spec = 10.0 * np.log10(np.maximum(features_specgram, 1e-10)).T
fig, ax = plt.subplots(1, 1)
ax.set_ylabel('Frequency (Hz)')
ax.matshow(log_spec, origin="lower", aspect="auto")

xlim = ax.get_xlim()
xticks = ax.get_xticks()
xticklabels = [f"{int(t * hop_length / 1000)}" for t in xticks]
ax.set_xticks(xticks)
ax.set_xticklabels(xticklabels)
# reset the xlim, which may have been modified by setting the xticks
ax.set_xlim(xlim)
ax.set_xlabel('Time (s)')

# y-axis is frequency
ylim = ax.get_ylim()
yticks = ax.get_yticks()
# convert yticks into frequencies
frequencies = yticks / nfft * audio.sample_rate
frequencies = [f"{int(f)}" for f in frequencies]
ax.set_yticks(yticks)
ax.set_yticklabels(frequencies)
ax.set_ylim(ylim)

# Display the image
plt.show()
